<a href="https://colab.research.google.com/github/jmlDC/Truth-Thesis22-23/blob/Scraping-trial/ScrapingNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install requests-html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.4 MB/s 
     |████████████████████████████████| 140 kB 42.3 MB/s 
     |████████████████████████████████| 112 kB 63.4 MB/s 
     |████████████████████████████████| 127 kB 65.9 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=9d8b5d966e9042fe27d8ae8804e0359501a02d0df856b6cf2a513032f3ce99bc
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=e1ba7c3ac96454313ff196f613e25e16ee493dedc31f1e246f1e21cd3bf9f761
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

from requests_html import HTMLSession 



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dir  = "/content/gdrive/MyDrive/THESIS-MS/Truth-Thesis22-23/"


In [ ]:
## Politifact 
pagesToGet= 1
upperframe=[]  
site = "politifact"


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.

    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))

    
    filename= f"{dir + site}NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
data.head()

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30


,Statement,Link,Date,Source,Label
0,Video shows there weren’t Marines present duri...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Viral image,false
1,“The Russian army is withdrawing from Ukraine!...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Facebook posts,false
2,"“Here in Nevada, the DMV registers everybody (...",https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Jim Marchant,false
3,Executive Order 14067 will allow the federal g...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",TikTok posts,false
4,Queen Elizabeth died because of the COVID-19 v...,https://www.politifact.com/factchecks/2022/sep...,"ember 8, 2022",Tweets,false


In [ ]:
def extract_source(url):
    agent = {"User-Agent":"Chrome/105.0.0.0"}
    try:
      source=requests.get(url, headers=agent)
    except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
    
    return source                                               #ignore this page. Abandon this and go back.



In [ ]:
## Manila bulletin
pagesToGet= 1
upperframe=[]  
topic = "ukraine war"
site = "manilaBulletin"

data=pd.DataFrame(columns=['Statement','Link','Date',"Topic"])


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://mb.com.ph/page/'+str(page)+'/?s='+topic.replace(" ", "+")
    print(url)

    time.sleep(2)   
    soup=BeautifulSoup(extract_source(url).text, 'html.parser')

    links=soup.find("article", {'class' : 'content-bar col-12 col-lg-12 order-1 col-xl order-xl-2'}).find_all('li',attrs={'class':'article'})
    print(len(links))
    
    filename= f"{dir + site}NEWS.csv"    
    for j in links:
        frame=[]
        try:
          Statement = j.find('h4').find('a').text
          Link = j.find('h4',attrs={'class':'title'}).find('a')['href'].strip()
          Date = j.find('time', {'class': 'time-ago'})['data-time'][0:10].strip()
          frame.extend((Statement, Link, Date, topic))
        except Exception as e:
            pass

        data = data.append(pd.Series(frame, index=data.columns[:len(frame)]), ignore_index=True)
        # upperframe.extend(frame)
data.to_csv(filename)
data.head()

processing page : 1
https://mb.com.ph/page/1/?s=ukraine+war
16
processing page : 2
https://mb.com.ph/page/2/?s=ukraine+war
16
processing page : 3
https://mb.com.ph/page/3/?s=ukraine+war
16


,Statement,Link,Date,Topic
0,Gatchalian seeks impact assessment of Russia-U...,https://mb.com.ph/2022/08/04/gatchalian-seeks-...,2022-08-04,ukraine war
1,"US sanctions Putin ‘girlfriend,’ more oligarch...",https://mb.com.ph/2022/08/03/us-sanctions-puti...,2022-08-03,ukraine war
2,Pacquiao-Yoo exhibition match to serve as char...,https://mb.com.ph/2022/07/20/pacquiao-yoo-exhi...,2022-07-20,ukraine war
3,Ukraine war to dominate G20 finance chief meet...,https://mb.com.ph/2022/07/15/ukraine-war-to-do...,2022-07-15,ukraine war
4,Ukraine war could boost illegal drug productio...,https://mb.com.ph/2022/06/27/ukraine-war-could...,2022-06-27,ukraine war


In [116]:
data2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])

for index, row in data.head(3).iterrows():
  url = row["Link"] 
  print(url)

  time.sleep(2)   
  soup=BeautifulSoup(extract_source(url).text, 'html.parser')

  content=soup.find("article", {'id' : 'the-article-content'})

  filename= f"{dir+site+'-'+topic}NEWS.csv"
  
  for j in content:
      row=[]
      try:
        Title = j.find('h2',{'class':'title'}).text.strip()
        # print(Title)
        # row.append(Title)
        Author = j.find('p',attrs={'class':'author'}).find('a').text[3:].strip()
        # print(Author)
        Date = j.find('p', attrs={'class': 'published'}).text[10:-9].strip()
        # print(Date)
        Text = j.find('section', attrs={'class':'article-content'}).find_all("p").text.strip()
        print(Text)
        row.extend((Title,Author,Date, Text))
        print("r:", row)
      except Exception as e:
          error_type, error_obj, error_info = sys.exc_info()      # get the exception information
          print (error_type, 'Line:', error_info.tb_lineno)
          pass
      # print(frame)
  data2 = data2.append(pd.Series(row, index=data.columns[:len(row)]), ignore_index=True)
  
      
data2.head()

https://mb.com.ph/2022/08/04/gatchalian-seeks-impact-assessment-of-russia-ukraine-war-on-ph/
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 24
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 17
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 17
<class 'TypeError'> Line: 17
https://mb.com.ph/2022/08/03/us-sanctions-putin-girlfriend-more-oligarchs-for-complicity-in-ukraine-war/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 24
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 17
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 17
<class 'TypeError'> Line: 17
https://mb.com.ph/2022/07/20/pacquiao-yoo-exhibition-match-to-serve-as-charity-event-for-ukraine-war-victims/
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 24
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 17
<class 'TypeError'> Line: 17
<class 'AttributeError'> Line: 17
<class 'TypeError'> Line: 17


,Title,Author,Date,Text
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN


In [113]:
print(content)

<article class="article" id="the-article-content">
<header class="row article-header">
<div class="col-12 col-md">
<div class="breadcrumbs">
<span property="itemListElement" typeof="ListItem"><a class="taxonomy category" href="https://mb.com.ph/category/sports/" property="item" title="Go to the Sports category archives." typeof="WebPage"><span property="name">Sports</span></a><meta content="1" property="position"/></span> / <span property="itemListElement" typeof="ListItem"><a class="taxonomy category" href="https://mb.com.ph/category/sports/boxing/" property="item" title="Go to the Boxing category archives." typeof="WebPage"><span property="name">Boxing</span></a><meta content="2" property="position"/></span> / <span class="post post-post current-item">Pacquiao-Yoo exhibition match to serve as charity event for Ukraine war victims</span></div> </div>
<div class="col-12 col-md-auto">
<div class="share-count">
<div class="d-flex a2a_kit a2a_kit_size_32 a2a_square_style" data-a2a-title="